# Importamos librerías y cargamos la base de datos para comenzar a analizar

In [ ]:
import pandas as pd
import numpy as np
import re
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv('properatti.csv', index_col='Unnamed: 0')

In [ ]:
data.head(3)

 Vemos cuantas Filas y Columnas tiene

In [ ]:
print("cantidad de filas: " + str(data.shape[0]))
print("cantidad de columnas: " + str(data.shape[1]))

In [ ]:
data.info()

In [ ]:
data.describe()

Eliminamos columnas que sabemos que no vamos a utilizar.

In [ ]:
data.drop(["image_thumbnail", "properati_url", "expenses", "floor"], axis=1, inplace = True)

### Obtenemos los porcentajes de datos faltantes de cada columna

In [ ]:
def freq_na(df, zeros=False):
    miss      = df.isnull().sum(axis=0)
    miss_prop = miss / len(df)
    freq = pd.concat([miss, miss_prop], axis=1)
    freq.columns = ['total', 'proporcion']
    return freq if zeros else freq[freq['total'] > 0].sort_values(by='proporcion',ascending=False)

freq_na(data)

In [ ]:
def mostrar_nulos(df):
    
    nulos = df.isnull().sum() / len(df)
    nulos = nulos[nulos > 0]
    nulos.sort_values(inplace=True)
    
    nulos = nulos.to_frame()
    nulos.columns = ['total']
    nulos.index.names = ['variable']
    nulos['variable'] = nulos.index
    # ploteo
    sns.set(style="whitegrid", color_codes=True)
    sns.barplot(x='variable', y='total', data=nulos)
    plt.xticks(rotation = 90)
    plt.show()
mostrar_nulos(data)

CONCLUSIÓN DE LA OBSERVACIÓN INICIAL:

El DataSet posee algunas columnas con muchos valores faltantes, posibles duplicados o posibles datos no confiables.

Las columnas o datos centrales del análisis que realizaremos son aquellos que nos indiquen:

a) ubicación de las observaciones

b) precio x M2

c) las variables de las cuales dependan las mismas (como ser tipo de propiedad, amenities, garage).

Por lo antes dicho, en este momento sólo nos limitaremos a realizar un drop de las observaciones duplicadas que puedan existir.

##### Evaluando data para ver si hay datos duplicados incluyendo la mayor cantidad de columnas que logicamente deban coincidir.

In [ ]:
data_duplicates_mask = data.duplicated(subset=["place_name", "country_name", "description", "title", "lat-lon", "state_name", "price_aprox_usd"], keep="first")
print("registros duplicados en data: ", any(data_duplicates_mask))
print("cantidad de registros duplicados en data: ", data_duplicates_mask.sum())

In [ ]:
data= data.drop_duplicates(subset=["place_name", "country_name", "description", "title", "lat-lon", "state_name", "price_aprox_usd"], keep="first")
data.shape

### ANÁLISIS DE COLUMNAS DE 0 A 9 - UBICACION DE LAS OBSERVACIONES

##### Columna operation

In [ ]:
# Como vemos posee la totalidad de valores sell y sin vacíos, por lo que no agregaría valor. Atacaremos el Drop de columnas más adelante.
operation_group = data.groupby('operation').country_name.count()
operation_group

##### Columna property type

In [ ]:
# Como vemos posee la totalidad de valores completos y nos será de mucha importancia para analizar los valores por tipo de propiedad.
property_type_group = data.groupby('property_type').property_type.count()
property_type_group

Todos los valores estan son "PH", "apartment", "house" y "store". Sin valores nulos.

In [ ]:
ciudades = data.groupby('place_name').place_name.count()
ciudades_ordenadas = ciudades.sort_values(ascending=False)
ciudades_ordenadas.head(10)

##### Columna place_name

In [ ]:
data.place_name.isnull().sum()

Contiene 23 valores nulos.

In [ ]:
place_name_nulos = data.loc[data.place_name.isnull(), :]
place_name_nulos

Todos los valores nulos son de Tigre.

In [ ]:
data.place_name.fillna('Tigre', inplace = True)

In [ ]:
data.place_name.isnull().sum()

In [ ]:
data.place_name[6489]

Reemplazo de todos los valores nulos por "Tigre", ya que en place_with_parent_names contiene en todos 'Tigre' 

##### Columna country_name

In [ ]:
# Como vemos posee la totalidad de valores Argentina, por lo que no agregaría valor. Atacaremos el Drop de columnas más adelante.
country_name_group = data.groupby('country_name').country_name.count()
country_name_group

Todos los valores son "Argentina" y no hay vacios.

##### Columna state_name

In [ ]:
state_name_group = data.groupby('state_name').state_name.count()
state_name_group

Ningun valor nulo. Algunas provincias poseen pocas observaciones, cosa que analizaremos más adelante

##### Columna place_with_parent_names

In [ ]:
# Analizamos esta columna que como vemos abajo tiene separado por una barra vertical cada ubicación de más general a menos general.
# Parte del país, luego va a la provincia, luego la localidad y el barrio en algunos casos.

place_with_parent_names_group = data.groupby('place_with_parent_names').place_with_parent_names.count()
place_with_parent_names_group.index

In [ ]:
print("Contiene", len(place_with_parent_names_group.index),"valores distintos")

In [ ]:
#Separamos la columna por la barra verical para poder aperturarlo.

placestr=data.place_with_parent_names.str
placeseparado = placestr.split('|', expand=True)
placeseparado

In [ ]:
# Renombramos las columnas del nuevo DF para luego comparar. Tener en cuenta que place_name tiene la mínima expresión de la ubicación.

placeseparado.rename(columns={1:'country_name_nuevo', 2:'state_name_nuevo', 3:'place_name_nuevo1', 4:'place_name_nuevo2'},inplace=True)

In [ ]:
placeseparado.info()

In [ ]:
#comparamos si coinciden el país que extrajimos de la columna con country_name. Está ok
controlpaises = data['country_name'] == placeseparado['country_name_nuevo']
controlpaises.value_counts()

In [ ]:
#comparamos ahora si coincide la provincia y está ok.
controlprovincias = data['state_name'] == placeseparado['state_name_nuevo']
controlprovincias.value_counts()

In [ ]:
# tratamos de analizar ahora place_name.
# vemos que la columna 6 del nuevo DF son vacios asique la podemos eliminar
placeseparado[6][placeseparado[6].notnull()].value_counts()

In [ ]:
# vemos que la columna 5 del nuevo DF la mayoría son vacíos y algunos son Barrios.
a = placeseparado[5][placeseparado[5].notnull()].value_counts()
print(a)

# transformamos en NaN los vacíos
placeseparado[5] = placeseparado[5].apply(lambda x: np.NaN if x=='' else x)
a = placeseparado[5][placeseparado[5].notnull()].value_counts()
print(a)

In [ ]:
data.place_name.value_counts()

In [ ]:
# De esta columna 5 del nuevo DF se puede ver que son barrios cerrados y coinciden con el data frame original.
b = data.place_name[placeseparado[5].notnull()]
print(b)

c = placeseparado[5][placeseparado[5].notnull()] == b
print(c.value_counts())

# Acá vemos que la totalidad de esos Barrios corresponden a Nordelta, por lo que los sacamos del Data Frame original
# y reemplazamos todos esos valores por Nordelta para poder analizarlos mejor
d = placeseparado['place_name_nuevo2'][placeseparado[5].notnull()]
print(d.value_counts())

data.place_name[placeseparado[5].notnull()] == d
data.place_name.value_counts()

In [ ]:
# vemos que la columna 4 que llamamos place_name_nuevo2 del nuevo DF la mayoría son vacíos.
a = placeseparado.place_name_nuevo2[placeseparado['place_name_nuevo2'].notnull()].value_counts()
print(a)

# transformamos en NaN los vacíos
placeseparado.place_name_nuevo2 = placeseparado.place_name_nuevo2.apply(lambda x: np.NaN if x=='' else x)
a = placeseparado.place_name_nuevo2[placeseparado['place_name_nuevo2'].notnull()].value_counts()
print(a)

In [ ]:
# De esta columna 4 del nuevo DF se puede ver que todo coincide con place_name.
b = data.place_name[placeseparado['place_name_nuevo2'].notnull()]
print(b)

c = placeseparado.place_name_nuevo2[placeseparado['place_name_nuevo2'].notnull()] == b
print(c.value_counts())

d = placeseparado['place_name_nuevo1'][placeseparado['place_name_nuevo2'].notnull()]
print(d.value_counts())

CONCLUSIÓN DEL ANALISIS DE LAS COLUMNAS DE UBICACIÓN:

El DataSet posee observaciones repartidas poro todo el país, y antes de sacar una conclusión, vamos a analizar el peso real de la cantidad de observaciones en cada territorio.

Como vemos abajo, son 28 las zonas provinciales (incluye la provincia de Bs As separada)

In [ ]:
# Vemos distribución por provincia
distr_x_pcia = data['state_name'].value_counts()
print(distr_x_pcia.count())
distr_x_pcia

Vamos a listar el tamaño en KMS2 de las distintas provincias (o zonas) de las cuales tenemos datos y a calcular la cantidad de operaciones por KM2 que tenemos en cada una

#Capital Federal                 202
#Bs.As. G.B.A. Zona Norte        1427
#Bs.As. G.B.A. Zona Sur          1207
#Córdoba                         165321
#Santa Fe                        133007
#Buenos Aires Costa Atlántica    1280
#Bs.As. G.B.A. Zona Oeste        1046
#Buenos Aires Interior           302611
#Río Negro                       203013
#Neuquén                         94078
#Mendoza                         148827
#Tucumán                         22524
#Corrientes                      88199
#Misiones                        29801
#Entre Ríos                      78781
#Salta                           155488
#Chubut                          224686
#San Luis                        76748
#La Pampa                        143440
#Formosa                         72066
#Chaco                           99633
#San Juan                        89651
#Tierra Del Fuego                21478
#Catamarca                       102602
#Jujuy                           53219
#Santa Cruz                      243943
#Santiago Del Estero             136351
#La Rioja                        89680

In [ ]:
tamañokms = pd.Series([202, 1427, 1207, 165321, 133007, 1280, 1046, 302611, 203013, 94078, 148827, 22524, 88199, 29801, 78781, 155488, 224686, 76748, 143440, 72066, 99633, 89651, 21478, 102602, 53219, 243943, 136351, 89680])
print(distr_x_pcia.shape)
print(tamañokms.shape)
print(distr_x_pcia.dtype)
print(tamañokms.dtype)

print("Capital Federal - Operaciones por KM2 -: ", (distr_x_pcia[0]/tamañokms[0]).round(2))
print("Bs.As. G.B.A. Zona Norte - Operaciones por KM2 -: ", (distr_x_pcia[1]/tamañokms[1]).round(2))
print("Bs.As. G.B.A. Zona Sur - Operaciones por KM2 -: ", (distr_x_pcia[2]/tamañokms[2]).round(2))
print("Córdoba - Operaciones por KM2 -: ", (distr_x_pcia[3]/tamañokms[3]).round(2))
print("Santa Fe - Operaciones por KM2 -: ", (distr_x_pcia[4]/tamañokms[4]).round(2))
print("Buenos Aires Costa Atlántica - Operaciones por KM2 -: ", (distr_x_pcia[5]/tamañokms[5]).round(2))
print("Bs.As. G.B.A. Zona Oeste - Operaciones por KM2 -: ", (distr_x_pcia[6]/tamañokms[6]).round(2))
print("Buenos Aires Interior - Operaciones por KM2 -: ", (distr_x_pcia[7]/tamañokms[7]).round(2))
print("Río Negro - Operaciones por KM2 -: ", (distr_x_pcia[8]/tamañokms[8]).round(2))
print("Neuquen - Operaciones por KM2 -: ", (distr_x_pcia[9]/tamañokms[9]).round(2))
print("Mendoza - Operaciones por KM2 -: ", (distr_x_pcia[10]/tamañokms[10]).round(2))
print("Tucuman - Operaciones por KM2 -: ", (distr_x_pcia[11]/tamañokms[11]).round(2))
print("Corrientes - Operaciones por KM2 -: ", (distr_x_pcia[12]/tamañokms[12]).round(2))
print("Misiones - Operaciones por KM2 -: ", (distr_x_pcia[13]/tamañokms[13]).round(2))
print("Entre Ríos - Operaciones por KM2 -: ", (distr_x_pcia[14]/tamañokms[14]).round(2))
print("Salta - Operaciones por KM2 -: ", (distr_x_pcia[15]/tamañokms[15]).round(2))
print("Chubut - Operaciones por KM2 -: ", (distr_x_pcia[16]/tamañokms[16]).round(2))
print("San Luis - Operaciones por KM2 -: ", (distr_x_pcia[17]/tamañokms[17]).round(2))
print("La Pampa - Operaciones por KM2 -: ", (distr_x_pcia[18]/tamañokms[18]).round(2))
print("Formosa - Operaciones por KM2 -: ", (distr_x_pcia[19]/tamañokms[19]).round(2))
print("Chaco - Operaciones por KM2 -: ", (distr_x_pcia[20]/tamañokms[20]).round(2))
print("San Juan - Operaciones por KM2 -: ", (distr_x_pcia[21]/tamañokms[21]).round(2))
print("Tierra del Fuego - Operaciones por KM2 -: ", (distr_x_pcia[22]/tamañokms[22]).round(2))
print("Catamarca - Operaciones por KM2 -: ", (distr_x_pcia[23]/tamañokms[23]).round(2))
print("Jujuy - Operaciones por KM2 -: ", (distr_x_pcia[24]/tamañokms[24]).round(2))
print("Santa Cruz - Operaciones por KM2 -: ", (distr_x_pcia[25]/tamañokms[25]).round(2))
print("Santiago del Estero - Operaciones por KM2 -: ", (distr_x_pcia[26]/tamañokms[26]).round(2))
print("La Rioja - Operaciones por KM2 -: ", (distr_x_pcia[27]/tamañokms[27]).round(2))



In [ ]:
# Vemos que dentro de Capital Federal, 1027 valores no tienen definición de Localidad, por lo que estas tendremos que buscarlas por el lado de lat-lon.
# Por otro lado vemos que Palermo está divido por zonas, en esta primera iteración lo dejaremos así.
Control = data.place_name[data['state_name'] == 'Capital Federal']
print(Control.value_counts())
print(Control.shape)
Control.value_counts().head(40)

In [ ]:
# Vemos que dentro de GBA Zona Norte, 221 valores no tienen definición de Localidad, por lo que al no ser significativas las eliminaremos.
Control = data.place_name[data['state_name'] == 'Bs.As. G.B.A. Zona Norte']
print(Control.value_counts())
print(Control.shape)
Control.value_counts().head(40)

CONCLUSIÓN DEL ANALISIS DE LAS COLUMNAS DE UBICACIÓN: Si bien es correcto que en el Interior existe mucho terreno con poca densidad poblacional y podríamos analizar ciudades puntuales como Santa Fe, Rosario o Córdoba, en esta primera iteración vamos a limitar el análisis a las 2 zonas con mayor cantidad de operaciones por KM2 --> Capital Federal y GBA Zona Norte.

CREAMOS UN DATAFRAME EN EL QUE SÓLO CONTIENE A CABA Y GBA ZONA NORTE

In [ ]:
cf = data.loc[:, 'state_name'].isin(['Capital Federal','Bs.As. G.B.A. Zona Norte'])
data = data.loc[cf]
data.head(5)

In [ ]:
place_split = data.place_with_parent_names.str.split('|', expand=True).rename({1:'pais', 2:'provincia', 3:'localidad', 4:'barrio'}, axis=1).drop([0,5,6], axis=1)
place_split['geonames_id'] = data['geonames_id']
place_split

In [ ]:
localidades = place_split.groupby('localidad').localidad.count().sort_values(ascending=False)
localidades.head(20)

In [ ]:
place_split['localidad'] = place_split['localidad'].apply(lambda x: np.NaN if x=='' else x)

In [ ]:
mascara_sin_localidad = place_split.localidad.isnull()
mascara_sin_localidad.sum()

In [ ]:
mascara_sin_localidad = place_split.localidad.isnull()
geonames_id_sin_localidad = place_split[mascara_sin_localidad].groupby('geonames_id').geonames_id.count().sort_values(ascending=False)
geonames_id_sin_localidad

In [ ]:
repeticiones_geonames_id_sin_localidad = place_split[place_split.geonames_id == 3433955.0].groupby('geonames_id').geonames_id.count().sort_values(ascending=False)
repeticiones_geonames_id_sin_localidad

In [ ]:
repeticiones_geonames_id_sin_localidad = place_split[place_split.geonames_id == 3435907.0].groupby('geonames_id').geonames_id.count().sort_values(ascending=False)
repeticiones_geonames_id_sin_localidad

Podemos ver que los dos geonames_id que nos faltan en ningun registro tienen imputado la localidad.

In [ ]:
mascara_capital = place_split.provincia == 'Capital Federal'

In [ ]:
mascara_bs_as_nor = place_split.provincia == 'Bs.As. G.B.A. Zona Norte'

In [ ]:
lat_lon_sin_localidad_capital = data[mascara_sin_localidad].groupby('lat-lon').lat.count().sort_values(ascending=False)
lat_lon_sin_localidad_capital

In [ ]:
print("Hay", lat_lon_sin_localidad_capital.sum(), "operaciones en Capital Federal sin localidad que tienen imputado lat-lon. Se repiten asi que no son datos muy fiables")

In [ ]:
lat_lon_sin_localidad_bs_as_nor = data[mascara_sin_localidad&mascara_bs_as_nor].groupby('lat').lat.count().sort_values(ascending=False)
lat_lon_sin_localidad_bs_as_nor

In [ ]:
print("Hay", lat_lon_sin_localidad_bs_as_nor.sum(), "operaciones en Bs.As. G.B.A. Zona Norte sin localidad que tienen imputado lat-lon.")

### ANÁLISIS DE COLUMNAS DE 10 A 17 - PRECIO POR METRO CUADRADO

#### ANÁLISIS DE PRECIOS

In [ ]:
# Vamos a analizar puntualmente la columna price
# Vemos que es de tipo flotante y posee en principio los valores a los que se realizó cada operación. 
data['price']

In [ ]:
# Si vemos la columna siguiente (tipo Str), posee la moneda en la que se realizó la operación.
data['currency']

In [ ]:
# Vemos que de los 54887 valores, 3180 son nulos para la columna de price
data.price.isnull().sum()

In [ ]:
# Vemos también que 3180 son nulos para la columna de currency
data.currency.isnull().sum()

In [ ]:
# Vamos a asegurarnos que los no nulos coincidan entre las columnas
price_nulo_bool = data.price.isnull()
currency_nulo_bool = data.currency.isnull()
precio_sin_currency = price_nulo_bool == currency_nulo_bool
precio_sin_currency.value_counts()

In [ ]:
# En el DF original había valores 0 en la columna Price, por lo que vamos a corroborar que ahora no exista ninguno.

precios0 = data['price'] == 0
precios0.value_counts()

In [ ]:
#Ahora vamos a ver las otras columnas si también coinciden
data.price_aprox_local_currency.isnull().sum()

In [ ]:
data.price_aprox_usd.isnull().sum()

In [ ]:
# nos aseguramos que los valores de estas 3 columnas coincidan en los nulos
price_nulo1_bool = data.price_aprox_local_currency.isnull()
price_nulo2_bool = data.price_aprox_usd.isnull()
price_nulo_bool = data.price.isnull()
precio_control1 = price_nulo1_bool == price_nulo2_bool
precio_control2 = price_nulo1_bool == price_nulo_bool
a = precio_control1.value_counts()
print(a)
b = precio_control2.value_counts()
print(b)

In [ ]:
# Vemos que en la columna currency, actualmente hay 3 tipos de monedas, por lo que analizaremos 1 por 1 (el UYU fue eliminado por zonas, pero el tipo de cambio también era lógico).
data['currency'].value_counts()

In [ ]:
#Vemos los valores de moneda PEN (Sol Peruano)
CurrencyPEN = data.loc[:, "currency"] == "PEN"
dfCurrencyPEN = data.loc[CurrencyPEN]
dfCurrencyPEN.loc[:, ['place_name','country_name', 'state_name', 'price', 'currency', 'price_aprox_local_currency', 'price_aprox_usd']]

In [ ]:
# Ok, es logico el tipo de cambio del PEN por lo que lo dejamos así
print(117139/380000)
print(292848/950000)

In [ ]:
#Vemos los valores de moneda ARS (Peso Argentino)
CurrencyARS = data.loc[:, "currency"] == "ARS"
dfCurrencyARS = data.loc[CurrencyARS]
dfCurrencyARS.loc[:, ['place_name','country_name', 'state_name', 'price', 'currency', 'price_aprox_local_currency', 'price_aprox_usd']]

In [ ]:
# Vemos que entre Price y price_aprox_local_currency no son exactamente los mismos porque seguramente la de local toma un promedio
# del tipo de cambio, pero al menos la división entre una y otra columna debería tender a 1
controlpricears = dfCurrencyARS.price / dfCurrencyARS.price_aprox_local_currency
may = controlpricears > 1.05
men = controlpricears < 0.95
print(may.value_counts())
men.value_counts()
# Vemos que la diferencia en todos los casos está entre 5% mayor o menor, asique estaría ok

In [ ]:
#Ahora revisaremos el Tipo de Cambio Usd/Ars
controltipodecambio = dfCurrencyARS.price / dfCurrencyARS.price_aprox_usd
print(controltipodecambio.max())
print(controltipodecambio.min())
tipo_cambio_ars_promedio = controltipodecambio.mean()
tipo_cambio_ars_promedio
# No tenemos las fechas de las operaciones, pero la información se corresponde con tipos de cambio del año 2017 lo que es posible
# Por otro lado no existen tipos de cambio fuera de lo común, son todos lógicos.

In [ ]:
#Vemos los valores de moneda USD (Dolar Americano)
CurrencyUSD = data.loc[:, "currency"] == "USD"
dfCurrencyUSD = data.loc[CurrencyUSD]
dfCurrencyUSD.loc[:, ['place_name','country_name', 'state_name', 'price', 'currency', 'price_aprox_local_currency', 'price_aprox_usd']]

In [ ]:
# Debería coincidir la columna Price con price_aprox_usd asique vamos a controlarlo
controlvaloresenusd = dfCurrencyUSD.price - dfCurrencyUSD.price_aprox_usd
controlvaloresenusd.value_counts()
# Vemos que todos coinciden asique perfecto.

CONCLUSIÓN INICIAL PRECIOS: Consideramos que los valores incluidos en las columnas analizadas son lógicos y sólidos para ahora avanzar con la búsqueda de datos faltantes u Outliers.

Búsqueda de valores faltantes:

In [ ]:
# recordemos que esta columna tiene 54887 datos de los cuales 3180 datos son NaN (5,79%)
is_null_result = data.price_aprox_usd.isnull()
is_notnull_result = data.price_aprox_usd.notnull()
totaldata=data.price_aprox_usd.shape[0]
print(totaldata)
cant_notnull = is_notnull_result.sum()
print(cant_notnull)
cant_nulls = is_null_result.sum()
print(cant_nulls)
print(cant_nulls/totaldata*100)

In [ ]:
# de una observación visual podemos ver que en la columna "Title" podemos sacar valores de publicacion que si bien pueden ser distintos
# al valor de la operación final, no deberían distar demasiado de esta

#para los valores dolares entre 1000 y 999999
import re
pattern_dol = "(D *)(?P<Dolares1>[1-9]\d\d*).(?P<Dolares2>\d\d\d)"
pattern_dol_regex =  re.compile(pattern_dol)

resultado_dol = data.title.apply(lambda x: pattern_dol_regex.search(x))
dol_match1 = resultado_dol.apply(lambda x: x if x is None else x.group("Dolares1"))
dol_match2 = resultado_dol.apply(lambda x: x if x is None else x.group("Dolares2"))
dol_match = dol_match1 + dol_match2

dol_match_fill = dol_match.fillna(0)
dol_match_fill_numeric = dol_match_fill.astype(float)

dol_match_fill_numeric.value_counts()

In [ ]:
#para los valores dolares entre mayores a 999999
import re
pattern_dol = "(D *)(?P<Dolares1>[1-9]\d*).(?P<Dolares2>\d\d\d).(?P<Dolares3>\d\d\d)"
pattern_dol_regex =  re.compile(pattern_dol)

resultado_dol = data.title.apply(lambda x: pattern_dol_regex.search(x))
dol_match1 = resultado_dol.apply(lambda x: x if x is None else x.group("Dolares1"))
dol_match2 = resultado_dol.apply(lambda x: x if x is None else x.group("Dolares2"))
dol_match3 = resultado_dol.apply(lambda x: x if x is None else x.group("Dolares3"))
dol_match = dol_match1 + dol_match2 + dol_match3

dol_match_fill = dol_match.fillna(0)
dol_match_fill_numericmillon = dol_match_fill.astype(float)

dol_match_fill_numericmillon.value_counts()

In [ ]:
#para los valores pesos mayores a 999999 y luego lo pasamos a Dolares según el promedio del T/C
import re
pattern_pes = "[$] (?P<Pesos1>[1-9]\d*).(?P<Pesos2>\d\d\d).(?P<Pesos3>\d\d\d)"
pattern_pes_regex =  re.compile(pattern_pes)

resultado_pes = data.title.apply(lambda x: pattern_pes_regex.search(x))
pes_match1 = resultado_pes.apply(lambda x: x if x is None else x.group("Pesos1"))
pes_match2 = resultado_pes.apply(lambda x: x if x is None else x.group("Pesos2"))
pes_match3 = resultado_pes.apply(lambda x: x if x is None else x.group("Pesos3"))
pes_match = pes_match1 + pes_match2 + pes_match3

pes_match_fill = pes_match.fillna(0)
pes_match_fill_numericmillon = pes_match_fill.astype(float)

pes_match_fill_numericmillonDOLAR = pes_match_fill_numericmillon / tipo_cambio_ars_promedio
pes_match_fill_numericmillonDOLAR.value_counts()

In [ ]:
#para los valores pesos menosres a 999999 y luego lo pasamos a Dolares según el promedio del T/C
import re
pattern_pes = "[$] (?P<Pesos1>[1-9]\d\d\d*).(?P<Pesos2>\d\d\d)"
pattern_pes_regex =  re.compile(pattern_pes)

resultado_pes = data.title.apply(lambda x: pattern_pes_regex.search(x))
pes_match1 = resultado_pes.apply(lambda x: x if x is None else x.group("Pesos1"))
pes_match2 = resultado_pes.apply(lambda x: x if x is None else x.group("Pesos2"))
pes_match = pes_match1 + pes_match2

pes_match_fill = pes_match.fillna(0)
pes_match_fill_numericmenosmillon = pes_match_fill.astype(float)

pes_match_fill_numericmenosmillonDOLAR = pes_match_fill_numericmenosmillon / tipo_cambio_ars_promedio
pes_match_fill_numericmenosmillonDOLAR.value_counts()

In [ ]:
#Ahora en la variable "dol_match_fill_numeric" están los dolares menores a 999.999 Usd
#Ahora en la variable "dol_match_fill_numericmillon" están los dolares mayores a 999.999 Usd
#Ahora en la variable "pes_match_fill_numericmillonDOLAR" están expresados en Usd los pesos mayores a 999.999 $
#Ahora en la variable "pes_match_fill_numericmenosmillonDOLAR" están expresados en Usd los pesos menores a 999.999 $

In [ ]:
data.price_aprox_usd.isnull().sum()

In [ ]:
# Vamos a cambiar en la columna Precio todos los NaN por los valores de dol_match_fill_numeric, tener en cuenta que quedan 0 NaN y les
# pone 0.
data.price_aprox_usd.fillna(value=dol_match_fill_numeric, inplace=True)

In [ ]:
a = data.price_aprox_usd.isnull().sum()

data.price_aprox_usd.value_counts()

In [ ]:
data.price_aprox_usd = data.price_aprox_usd.apply(lambda x: np.NaN if x==0 else x)

In [ ]:
data.price_aprox_usd.fillna(value=dol_match_fill_numericmillon, inplace=True)
data.price_aprox_usd.value_counts()

In [ ]:
data.price_aprox_usd = data.price_aprox_usd.apply(lambda x: np.NaN if x==0 else x)

In [ ]:
data.price_aprox_usd.fillna(value=pes_match_fill_numericmillonDOLAR, inplace=True)
data.price_aprox_usd.value_counts()

In [ ]:
data.price_aprox_usd = data.price_aprox_usd.apply(lambda x: np.NaN if x==0 else x)

In [ ]:
data.price_aprox_usd.fillna(value=pes_match_fill_numericmenosmillonDOLAR, inplace=True)
data.price_aprox_usd.value_counts()

In [ ]:
data.price_aprox_usd = data.price_aprox_usd.apply(lambda x: np.NaN if x==0 else x)

In [ ]:
data.price_aprox_usd.isnull().sum()

Con la búsqueda de valores en Title, pasamos de 3180 valores nulos a 2478.

#### ANÁLISIS DE METROS 2

In [ ]:
m2_pattern = "\s(?P<metros>\d{0,3}?[.]?\d*)\s?(?P<sufijo>m2|M2|metros|mts|m²)"
m2_regex =  re.compile(m2_pattern)
m2_match = data.description.apply(lambda x: x if x is np.NaN else m2_regex.search(x))
m2_match_mask = m2_match.notnull()
data.loc[m2_match_mask, "M2"] = m2_match[m2_match_mask].apply(lambda x: x.group("metros"))

In [ ]:
data["M2"].replace(to_replace = "", value= np.NaN, inplace = True)
data["M2"] = data["M2"].astype(float)

In [ ]:
#si solo tengo valor en M2, lo llevo a metros
data['metros1'] = data[(data['surface_total_in_m2'].isnull()) & (data['surface_covered_in_m2'].isnull()) & (data['M2'].notnull())]["M2"]
#si tengo valor en surface_covered_in_m2 lo llevo a metros
data['metros2'] = data[(data['surface_total_in_m2'].isnull()) & (data['surface_covered_in_m2'].notnull())]['surface_covered_in_m2']
#si tengo valor en surface_total_in_m2 lo llevo a metros
data['metros3'] = data[(data['surface_total_in_m2'].notnull()) & (data['surface_covered_in_m2'].isnull())]['surface_total_in_m2']
#si tengo covered y total, tomo total
data['metros4'] = data[(data['surface_total_in_m2'].notnull()) & (data['surface_covered_in_m2'].notnull())]['surface_total_in_m2'] 

data["metros1"].fillna(0, inplace=True)
data["metros2"].fillna(0, inplace=True)
data["metros3"].fillna(0, inplace=True)
data["metros4"].fillna(0, inplace=True)

#creo nueva columna de m2 calculados donde sumo los metros de cada condicion.
data['m2_calculated'] = data.apply(lambda x: x['metros1'] + x['metros2'] + x["metros3"] + x["metros4"], axis=1)
data["m2_calculated"].replace(to_replace = 0, value= np.NaN, inplace = True)

#df.drop(["metros1", "metros2", "metros3", "metros4", "M2", "surface_covered_in_m2", "surface_total_in_m2"], axis=1, inplace = True)
# Sabemos que algunos datos recuperados son erroneos como por ejemplo "a tantos metros de tal lugar", pero al ser outliers, se filtrarán a posterior

In [ ]:
print('Cantidad surface_total_in_m2 null en df inicial:', data["surface_total_in_m2"].isnull().sum())
print('Cantidad surface_covered_in_m2 null en df inicial:', data["surface_covered_in_m2"].isnull().sum())
print('Cantidad m2_calculated null en df trabajado:', data["m2_calculated"].isnull().sum())

In [ ]:
# EN ESTE MOMENTO TENEMOS LA COLUMNA "price_aprox_usd" CON LOS PRECIOS EN USD DEFINITIVOS
# Y TENEMOS LA COLUMNA "m2_calculated" CON LOS M2 DEFINITIVOS

#### CREAMOS COLUMNA DE USD/M2

In [ ]:
# Creamos columna con los datos de Usd/M2 según las columnas creadas por nosotros
data["USDxM2"] = data["price_aprox_usd"]/data["m2_calculated"]

In [ ]:
print('Cantidad total datos USD:', data["price_aprox_usd"].shape[0])
print('Cantidad nulos datos USD:', data["price_aprox_usd"].isnull().sum())
print('Cantidad total datos M2:', data["m2_calculated"].shape[0])
print('Cantidad nulos datos M2:', data["m2_calculated"].isnull().sum())
print('Cantidad total datos USDxM2:', data["USDxM2"].shape[0])
print('Cantidad nulos datos USDxM2:', data["USDxM2"].isnull().sum())

In [ ]:
data.dtypes

In [ ]:
# Ahora procedemos a eliminar todas las filas donde no tenemos Usd/m2 ya que es un dato clave que no vemos conveniente inferir.
print(data.shape)
data = data.dropna(subset=['USDxM2'], axis = 0) #meter implace para dejarlo en la base.
print(data.shape)

In [ ]:
# Vemos la existencia de Outliers
sns.set_style('darkgrid')
plt.figure(figsize=(7, 5))
sns.boxplot(data=data.state_name[data.property_type=='house'], x=data.state_name[data.property_type=='house'], y=data.USDxM2)

plt.xlabel("Ciudades"); plt.ylabel("Usd/M2");plt.title("Distribución Dólares por M2 por Ciudad")
plt.show()

In [ ]:
data.info()

In [ ]:
data_agrupado = data.groupby(['property_type', 'state_name'])
data_agrupado.describe()['USDxM2'].round(2)

In [ ]:
# DEBIDO A QUE ESTAMOS FILTRANDO DESDE STATE NAME EN VEZ DE DESDE LA LOCALIDAD, ACA ELEGIMOS DEL 25%, CUANTAS MUESTRAS MENORES VAMOS A TOMAR Y DEL 75% CUANTAS MUESTRAS MAYORES
MEN = 0.5
MAY = 1.5


In [ ]:
Norte_PH = data.USDxM2[(data.state_name=='Bs.As. G.B.A. Zona Norte') & (data.property_type=='PH')]
Norte_PH_min = Norte_PH > (1049.50 * MEN)
Norte_PH_max = Norte_PH < (2020.38 * MAY)
Norte_PH_fin = Norte_PH_min & Norte_PH_max
print(Norte_PH_fin.value_counts())
Norte_PH_finT = Norte_PH_fin[Norte_PH_fin==1]
Norte_PH_finT.value_counts()

In [ ]:
Cap_PH = data.USDxM2[(data.state_name=='Capital Federal') & (data.property_type=='PH')]
Cap_PH_min = Cap_PH > (1310.81 * MEN)
Cap_PH_max = Cap_PH < (2300.00 * MAY)
Cap_PH_fin = Cap_PH_min & Cap_PH_max
print(Cap_PH_fin.value_counts())
Cap_PH_finT = Cap_PH_fin[Cap_PH_fin==1]
Cap_PH_finT.value_counts()

In [ ]:
Norte_apart = data.USDxM2[(data.state_name=='Bs.As. G.B.A. Zona Norte') & (data.property_type=='apartment')]
Norte_apart_min = Norte_apart > (1812.50 * MEN)
Norte_apart_max = Norte_apart < (2929.76 * MAY)
Norte_apart_fin = Norte_apart_min & Norte_apart_max
print(Norte_apart_fin.value_counts())
Norte_apart_finT = Norte_apart_fin[Norte_apart_fin==1]
Norte_apart_finT.value_counts()

In [ ]:
Cap_apart = data.USDxM2[(data.state_name=='Capital Federal') & (data.property_type=='apartment')]
Cap_apart_min = Cap_apart > (2093.02 * MEN)
Cap_apart_max = Cap_apart < (3129.30 * MAY)
Cap_apart_fin = Cap_apart_min & Cap_apart_max
print(Cap_apart_fin.value_counts())
Cap_apart_finT = Cap_apart_fin[Cap_apart_fin==1]
Cap_apart_finT.value_counts()

In [ ]:
Norte_house = data.USDxM2[(data.state_name=='Bs.As. G.B.A. Zona Norte') & (data.property_type=='house')]
Norte_house_min = Norte_house > (887.37 * MEN)
Norte_house_max = Norte_house < (1795.16 * MAY)
Norte_house_fin = Norte_house_min & Norte_house_max
print(Norte_house_fin.value_counts())
Norte_house_finT = Norte_house_fin[Norte_house_fin==1]
Norte_house_finT.value_counts()

In [ ]:
Cap_house = data.USDxM2[(data.state_name=='Capital Federal') & (data.property_type=='house')]
Cap_house_min = Cap_house > (935.67 * MEN)
Cap_house_max = Cap_house < (2065.05 * MAY)
Cap_house_fin = Cap_house_min & Cap_house_max
print(Cap_house_fin.value_counts())
Cap_house_finT = Cap_house_fin[Cap_house_fin==1]
Cap_house_finT.value_counts()

In [ ]:
Norte_store = data.USDxM2[(data.state_name=='Bs.As. G.B.A. Zona Norte') & (data.property_type=='store')]
Norte_store_min = Norte_store > (952.38 * MEN)
Norte_store_max = Norte_store < (2608.80 * MAY)
Norte_store_fin = Norte_store_min & Norte_store_max
print(Norte_store_fin.value_counts())
Norte_store_finT = Norte_store_fin[Norte_store_fin==1]
Norte_store_finT.value_counts()

In [ ]:
Cap_store = data.USDxM2[(data.state_name=='Capital Federal') & (data.property_type=='store')]
Cap_store_min = Cap_store > (1595.59 * MEN)
Cap_store_max = Cap_store < (4200.41 * MAY)
Cap_store_fin = Cap_store_min & Cap_store_max
print(Cap_store_fin.value_counts())
Cap_store_finT = Cap_store_fin[Cap_store_fin==1]
Cap_store_finT.value_counts()

In [ ]:
parametro = data.USDxM2[(Norte_PH_finT.index) | (Cap_PH_finT.index) | (Norte_apart_finT.index) | (Cap_apart_finT.index) | (Norte_house_finT.index) | (Cap_house_finT.index) | (Norte_store_finT.index) | (Cap_store_finT.index)]
parametro

In [ ]:
cf = data.loc[parametro.index]
data = cf

In [ ]:
data.shape

In [ ]:
# Vemos como queda luego de la eliminación de los Outliers
sns.set_style('darkgrid')
plt.figure(figsize=(7, 5))
sns.boxplot(data=data.state_name[data.property_type=='house'], x=data.state_name[data.property_type=='house'], y=data.USDxM2)

plt.xlabel("Provincia"); plt.ylabel("Usd/M2");plt.title("Distribución Dólares por M2 por cada Casas")
plt.show()

In [ ]:
sns.set_style('darkgrid')
plt.figure(figsize=(7, 5))
sns.boxplot(data=data.state_name[data.property_type=='store'], x=data.state_name[data.property_type=='store'], y=data.USDxM2)

plt.xlabel("Provincia"); plt.ylabel("Usd/M2");plt.title("Distribución Dólares por M2 por para Tiendas")
plt.show()

In [ ]:
sns.set_style('darkgrid')
plt.figure(figsize=(7, 5))
sns.boxplot(data=data.state_name[data.property_type=='apartment'], x=data.state_name[data.property_type=='apartment'], y=data.USDxM2)

plt.xlabel("Provincia"); plt.ylabel("Usd/M2");plt.title("Distribución Dólares por M2 por para Departamentos")
plt.show()

In [ ]:
sns.set_style('darkgrid')
plt.figure(figsize=(7, 5))
sns.boxplot(data=data.state_name[data.property_type=='PH'], x=data.state_name[data.property_type=='PH'], y=data.USDxM2)

plt.xlabel("Provincia"); plt.ylabel("Usd/M2");plt.title("Distribución Dólares por M2 por para PH")
plt.show()

In [ ]:
data.pivot_table(index='state_name', columns='property_type', aggfunc={'USDxM2':'mean', 'price_aprox_usd':'count'})

### Análisis de Amenities

##### Búsqueda de Garage en description

In [ ]:
garage_pattern = "(?P<garage>cochera|garage|estacionamiento)"
garage_regex =  re.compile(garage_pattern)

garage_match = data.description.apply(lambda x: x if x is np.NaN else garage_regex.search(x))
garage_match_mask = garage_match.notnull()
data.loc[garage_match_mask, "Garage"] = 1
data["Garage"].fillna(0,inplace=True)
data.loc[:,"Garage"] = data.loc[:,"Garage"].astype(int)

##### Búsqueda de propiedades "A estrenar" en description

In [ ]:
estrenar_pattern = "(?P<estrenar>(a estrenar)|(departamento nuevo))"
estrenar_regex =  re.compile(estrenar_pattern)

estrenar_match = data.description.apply(lambda x: x if x is np.NaN else estrenar_regex.search(x))
estrenar_match_mask = estrenar_match.notnull()
data.loc[estrenar_match_mask, "Estrenar"] = 1
data["Estrenar"].fillna(0, inplace=True)
data.loc[:,"Estrenar"] = data.loc[:,"Estrenar"].astype(int)

##### Búsqueda de Pileta en description

In [ ]:
pileta_pattern = "(?P<pileta>pileta|piscina|picina|pisina)"
pileta_regex =  re.compile(pileta_pattern)

pileta_match = data.description.apply(lambda x: x if x is np.NaN else pileta_regex.search(x))
pileta_match_mask = pileta_match.notnull()
data.loc[pileta_match_mask, "Pileta"] = 1
data["Pileta"].fillna(0, inplace=True)
data.loc[:,"Pileta"] = data.loc[:,"Pileta"].astype(int)

##### Búsqueda de Balcón en description

In [ ]:
balcon_pattern = "(?P<balcon>balcon|balcones|balcón)"
balcon_regex =  re.compile(balcon_pattern)

balcon_match = data.description.apply(lambda x: x if x is np.NaN else balcon_regex.search(x))
balcon_match_mask = balcon_match.notnull()
data.loc[balcon_match_mask, "Balcon"] = 1
data["Balcon"].fillna(0, inplace=True)
data.loc[:,"Balcon"] = data.loc[:,"Balcon"].astype(int)

##### Búsqueda de Gimnasio en description

In [ ]:
gym_pattern = "(?P<gym>gym|gimnasio|gimnacio)"
gym_regex =  re.compile(gym_pattern)

gym_match = data.description.apply(lambda x: x if x is np.NaN else gym_regex.search(x))
gym_match_mask = gym_match.notnull()
data.loc[gym_match_mask, "Gimnasio"] = 1
data["Gimnasio"].fillna(0, inplace=True)
data.loc[:,"Gimnasio"] = data.loc[:,"Gimnasio"].astype(int)

##### Búsqueda de Quincho en description

In [ ]:
quincho_pattern = "(?P<quincho>quincho|kincho|qincho|(\ssum\s))"
quincho_regex =  re.compile(quincho_pattern)

quincho_match = data.description.apply(lambda x: x if x is np.NaN else quincho_regex.search(x))
quincho_match_mask = quincho_match.notnull()
data.loc[quincho_match_mask, "Quincho"] = 1
data["Quincho"].fillna(0, inplace=True)
data.loc[:,"Quincho"] = data.loc[:,"Quincho"].astype(int)

Tanto Garage como Estrenar van a ser dos columnas separadas

La forma es imputación es a traves de variables dummies. Es decir, para cada amenitie agregamos una columna con el nombre de la misma y la completamos con 1 si el anuncio la indica y 0 en caso contrario.

In [ ]:
print("Total de \"{0}\" imputados: {1}".format("Quincho", data["Quincho"].sum()))
print("Total de \"{0}\" imputados: {1}".format("Gimnasio", data["Gimnasio"].sum()))
print("Total de \"{0}\" imputados: {1}".format("Balcon", data["Balcon"].sum()))
print("Total de \"{0}\" imputados: {1}".format("Pileta", data["Pileta"].sum()))
print("Total de \"{0}\" imputados: {1}".format("Garage", data["Garage"].sum()))
print("Total de \"{0}\" imputados: {1}".format("Estrenar", data["Estrenar"].sum()))
data['Amenities'] = data['Pileta'] + data['Quincho'] + data['Balcon'] + data['Gimnasio']

 1ER CONCLUSION LAS COLUMNAS DE PRICE POR M2 NO SE CONSIDERARAN PORQUE SE GENERARON A PARTIR DE DATOS DE SUPERFICIE Y PRECIO YA LIMPIOS Y ANALIZADOS
 
 3ER CONCLUSION LA COLUMNA ROOMS ES IMPORTANTE PORQUE DA VALOR AL DATASET, SE LLENARÀN LOS NULOS EN LA COLUMNA ROOMS.

#### Busqueda de Ambientes en title y description

In [ ]:
rooms_pattern = "\s(?P<ambientes>\d\d?)(\s?)(?P<sufijo>AMB|amb|Amb)"
rooms_regex =  re.compile(rooms_pattern)

rooms_match = data.description.apply(lambda x: x if x is np.NaN else rooms_regex.search(x))
rooms_match_mask = rooms_match.notnull()
data.loc[rooms_match_mask, "Ambientes"] = rooms_match[rooms_match_mask].apply(lambda x: x.group("ambientes"))

In [ ]:
data["Ambientes"] = data["Ambientes"].astype(float)
data.loc[(pd.isnull(data["rooms"])),"rooms"] = data["Ambientes"]
print('Cantidad rooms null en data inicial:', data["rooms"].isnull().sum())

Con el fin de buscar nulos por tipo de propiedad para ver cuanto peso tiene cada nulo por categorìa, generamos una mascara por columna analizada en esta seccion

In [ ]:
data['suptotnulo']= data.surface_total_in_m2.isnull()
data['supcubnulo']= data.surface_covered_in_m2.isnull()
data['ambnulo']= data.rooms.isnull()

In [ ]:
#aca hago una tabla con nulos discriminado por tipo de propiedad
data.groupby(['property_type']).aggregate({'suptotnulo': 'sum', 'supcubnulo': 'sum', 'ambnulo': 'sum'})

ANALISIS A: SUPERFICIE EN RELACION A LOS AMBIENTES


La idea es ver que por ejemplo los 2 ambientes tienen un rango de m2 y los 3 ambientes tambien se encuentran en cierto rango de m2, vamos a ver esa relacion 
graficando la relacion que hay entre cantidad de ambientes y superficie

In [ ]:
data.plot.scatter(x='Ambientes',y='m2_calculated', c='DarkBlue')

Conclusion: se pueden sacar outliers de propiedades de mas de 2000 m2 y propiedades de mas de 15 ambientes

In [ ]:
#superficie mayor a 10.000m2
data = data[data['m2_calculated'] < 2000]
data.shape

In [ ]:
#hago una mascara para graficar valores de menos de 13 ambientes
data = data[data['Ambientes'].fillna(0) <13]
data.shape

CONCLUSION, SE ELIMINAN DEL DATASET OUTLIERS DE INDICES DE MAS DE 15 AMBIENTES Y SUPERFICIES MAYORES A 2000M2

In [ ]:
ax2 = data.plot.scatter(x='Ambientes', y='m2_calculated', c='DarkBlue')

In [ ]:
data.drop(["metros1", "metros2", "metros3", "metros4", "M2", "Pileta", "Balcon", "Gimnasio", 
           "Quincho", "suptotnulo", "suptotnulo", "supcubnulo", "ambnulo"], axis=1, inplace = True)

In [ ]:
data.Ambientes.value_counts()

## CONCLUSIÓN: se genera un nuevo dataset final, el cual se va a exportar para el próximo análisis

Primero se eliminan las columnas que no se van a utilizar, se dejan las nuevas columnas que se generaron luego del análisis del dataset original

In [ ]:
data.drop(["operation", "country_name", 'place_with_parent_names',"geonames_id", 'lat', 'lon','currency',
           'price','price_aprox_local_currency','surface_total_in_m2','surface_covered_in_m2','price_usd_per_m2',
           'price_per_m2','rooms'], axis=1, inplace = True)

Luego se renombran las columnas, para generar el dataset final

In [ ]:
data.rename(columns={'property_type':'Tipo_Propiedad','place_name':'Barrio','state_name':'Región','price_aprox_usd':'Precio','description':'Descripción','title':'Título','m2_calculated':'Metros_cuadrados','USDxM2':'Precio_por_m2','Estrenar':'Estado'},inplace=True)


Se hace una revisión del nuevo dataset con un "info"

In [ ]:
data.info()

Finalmente exportamos el dataset para el análisis final

In [ ]:
data.to_csv('properati-fase2.csv', sep=',', index=False)